In [ ]:
import joblib
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd
import gym
from gym import spaces
import matlab.engine
from joblib import load
from stable_baselines3 import SAC
import matplotlib.pyplot as plt
from stable_baselines3.common.callbacks import BaseCallback
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy
from collections import deque

# Load the scaler
scaler = joblib.load('C:\\Users\\XuJ\\Jupyter-lab\\IUS_reinforcement_learning\\Scripts_required\\scaler.joblib')

# Start MATLAB engine
eng = matlab.engine.start_matlab()
eng.addpath('C:\\Users\\XuJ\\Jupyter-lab\\IUS_reinforcement_learning\\Scripts_required\\Complete_functions_reinforcement_learning')

# Load the pre-trained Python models
frequency_model = load('C:\\Users\\XuJ\\Documents\\Reinforcement_learning\\Gaussian Process_model_freq.joblib')
impedance_model = load('C:\\Users\\XuJ\\Documents\\Reinforcement_learning\\Gaussian Process_model.joblib')
print("Models loaded successfully")

class CustomCallback(BaseCallback):
    def __init__(self, verbose=0):
        super(CustomCallback, self).__init__(verbose)
        self.rewards = []
        self.frequencies = []
        self.impedances = []
        self.actions = []
        self.states = []
        self.timestep = 0  

    def _on_step(self):
        self.rewards.append(self.locals['rewards'][0])
        env = self.training_env.envs[0]
        
        # Save action
        self.actions.append(self.locals['actions'][0])
        
        # Save state (observation)
        self.states.append(self.locals['new_obs'][0])
        
        if env.frequency_history:
            self.frequencies.append(env.frequency_history[-1])
        else:
            self.frequencies.append(None)
        if env.impedance_slope_history:
            self.impedances.append(env.impedance_slope_history[-1])
        else:
            self.impedances.append(None)
        return True

class PMUTEnv(gym.Env):
    def __init__(self):
        super(PMUTEnv, self).__init__()
        self.action_space = spaces.Box(low=np.array([150, 80, -250, 0, 0, -250, 0]), 
                                       high=np.array([250, 100, -150, 250, 180, 0, 180]), dtype=np.float32)
        self.observation_space = spaces.Box(low=np.array([0.002, 0.65]), 
                                            high=np.array([0.009, 0.90]), 
                                            dtype=np.float32)
        self.frequency_history = []
        self.impedance_slope_history = []
        self.episode_observations = []
        self.episode_rewards = []
        self.max_steps = 50
        # Define feature names
        self.feature_names = [
            'polyarea', 'minboxarea', 'boxlongside', 'ellipsefitarea', 'ellipticity',
            'largestdistancevertices', 'circurity', 'mincirclearea', 'Sysmetry',
            'integrationpositive', 'integrationnegative', 'Peakvaluepositive',
            'Peakvaluenegative', 'Bandwidthpositive', 'Bandwidthnegative',
            'Periodicitycurvaturee', 'Yzeropoints', 'meanValue', 'medianValue',
            'standardDeviation', 'meanAbsoluteDeviation', 'quantile25', 'quantile75',
            'signalIQR', 'sampleSkewness', 'sampleKurtosis',
            'integrationpositive_Centroiddistance', 'Peakvaluepositive_Centroiddistance',
            'Peakvaluenegative_Centroiddistance', 'Bandwidthpositive_Centroiddistance',
            'Bandwidthnegative_Centroiddistance', 'meanValue_Centroiddistance',
            'medianValue_Centroiddistance', 'standardDeviation_Centroiddistance',
            'meanAbsoluteDeviation_Centroiddistance', 'quantile25_Centroiddistance',
            'quantile75_Centroiddistance', 'signalIQR_Centroiddistance',
            'sampleSkewness_Centroiddistance', 'sampleKurtosis_Centroiddistance'
        ]
        
        # Ensure feature names align with scaler
        if not hasattr(scaler, 'feature_names_in_'):
            raise ValueError("Scaler does not have feature names. Please use a scaler that preserves feature names.")
        if set(self.feature_names) != set(scaler.feature_names_in_):
            raise ValueError("Feature names do not match between environment and scaler.")
        
        # Reorder feature names to match scaler
        self.feature_names = list(scaler.feature_names_in_)
        
        # Default initial state
        self.initial_state = np.array([
            np.random.uniform(0.002, 0.009),  # frequency
            np.random.uniform(0.65, 0.90)     # impedance_slope
        ])
        self.current_state = self.initial_state.copy()  # To keep track of the current state

    def step(self, action):
        self.timestep += 1
        features = self.get_features(action)

        if np.all(features == 0):
            # If it's the first timestep (after increment), return [0, 0]
            if self.timestep == 1:
                return np.array([0, 0]), -1, True, {}
            # Otherwise return the history
            else:
                return np.array([self.frequency_history[-1], self.frequency_history[-1]]), -1, True, {}

        features = self.align_features(features)
        features[np.isnan(features)] = 0
        
        try:
            features_scaled = scaler.transform(features)
        except ValueError as e:
            print(f"Error during feature scaling: {e}")
            return None
        
        impedance_slope = impedance_model.predict(features_scaled)[0]
        frequency = frequency_model.predict(features_scaled)[0]
        self.frequency_history.append(frequency)
        self.impedance_slope_history.append(impedance_slope)

        observation = np.array([frequency, impedance_slope])
        reward = self.calculate_reward(frequency, impedance_slope)

        # Save the observation and reward
        self.episode_observations.append(observation)
        self.episode_rewards.append(reward)

        self.current_state = observation

        # Check if the episode should end
        done = abs(frequency - 0.0045) < 0.0001 and impedance_slope <= 0.78
        return observation, reward, done, {}

    def get_features(self, action):
        full_parameters = action.tolist()
        try:
            matlab_params = tuple(matlab.double([p]) for p in full_parameters)
            feature_table = eng.features_reinforcement02(*matlab_params, nargout=1)
            features = np.array(eng.table2array(feature_table))
            return features
        except Exception as e:
            print(f"MATLAB function failed with error: {e}")
            return None

    def align_features(self, features):
        features = features.squeeze()
        if features.shape[0] != len(self.feature_names):
            print(f"Warning: Number of features ({features.shape[0]}) does not match number of feature names ({len(self.feature_names)})")
            return None

        # Create a DataFrame with feature names
        features_df = pd.DataFrame([features], columns=self.feature_names)
        return features_df

    def calculate_reward(self, frequency, impedance_slope):
        # if self.timestep < 2000:
        #     # Phase 1: Focus on frequency
        #     frequency_reward = -abs(frequency - 0.005) * 2e5
        #     impedance_penalty = 0  # No penalty for impedance in this phase
        if self.timestep < 4000:
            # Phase 2: Balance frequency and impedance
            # frequency_reward = -abs(frequency - 0.0045) * 1e3
            # impedance_penalty = (0.90 - impedance_slope) / 5e-1
            frequency_reward = -abs(frequency - 0.0045) * 5e2
            impedance_penalty = 1/impedance_slope
        else:
            # Phase 3: Focus more on impedance
            frequency_reward = -abs(frequency - 0.0045) * 1e3  # Reduced focus on frequency
            impedance_penalty = (0.90 - impedance_slope) / 5e-1  # Increased focus on impedance
        
        return frequency_reward + impedance_penalty

    def reset(self):
        self.timestep = 0  # Reset timestep counter
        self.initial_state = np.array([
            np.random.uniform(0.002, 0.009),  # frequency
            np.random.uniform(0.65, 0.90)     # impedance_slope
        ])
        return self.initial_state

    def render(self, mode='human'):
        pass


# Create and wrap the environment
env = DummyVecEnv([lambda: PMUTEnv()])

# Define custom hyperparameters
custom_hyperparams = {
        "learning_rate": 5e-3,
        "batch_size": 128,
        "buffer_size": 100000,
        "learning_starts":5000,
        "gamma": 0.98,
        "tau": 0.01,
        "train_freq":1,
        "gradient_steps":4,
        "ent_coef":'auto',
        "policy_kwargs": dict(net_arch=[128, 128])
}

# Initialize the SAC agent with custom hyperparameters
model = SAC(
    "MlpPolicy",
    env,
    verbose=1,
    **custom_hyperparams
)

# Create the callback
callback = CustomCallback()

###### Train the agent, first phase
model.learn(total_timesteps=5000, callback=callback)

# Save the model
model.save("sac_pmut_model_50MHz_e")

# Load the saved model
model = SAC.load("sac_pmut_model_50MHz_e")

# Test function
def test_model_non_zero(model, env, n_steps=20, deterministic=True):
    test_rewards = []
    test_frequencies = []
    test_impedances = []
    test_actions = []  # New list to store actions
    obs = env.reset()
    step_count = 0
    while step_count < n_steps:
        action, _states = model.predict(obs, deterministic=deterministic)
        obs, rewards, dones, info = env.step(action)
        
        # Check if the result is non-zero
        # if rewards[0] != -10 :  # Assuming obs[0][0] is frequency and obs[0][1] is impedance slope
        #     test_rewards.append(rewards[0])
        #     test_frequencies.append(obs[0][0])
        #     test_impedances.append(obs[0][1])
        #     test_actions.append(action[0])  # Store the action
        #     step_count += 1
        test_rewards.append(rewards[0])
        test_frequencies.append(obs[0][0])
        test_impedances.append(obs[0][1])
        test_actions.append(action[0])  # Store the action
        step_count += 1
        if dones:
            obs = env.reset()
    
    return test_rewards, test_frequencies, test_impedances, test_actions

# Test with stochastic policy
stoch_rewards, stoch_frequencies, stoch_impedances, stoch_test_actions = test_model_non_zero(model, env, deterministic=False)

# Plotting
plt.figure(figsize=(15, 10))

# Training plots
plt.subplot(231)
plt.plot(callback.rewards)
plt.title('Training Rewards')
plt.xlabel('Step')
plt.ylabel('Reward')

plt.subplot(232)
plt.plot([f for f in callback.frequencies if f is not None])
plt.title('Training Frequencies')
plt.xlabel('Step')
plt.ylabel('Frequency')

plt.subplot(233)
plt.plot([i for i in callback.impedances if i is not None])
plt.title('Training Impedance Slopes')
plt.xlabel('Step')
plt.ylabel('Impedance Slope')


# Testing plots
plt.subplot(234)
plt.plot(stoch_rewards)
plt.title('Stochastic Policy Rewards (Non-Zero)')
plt.xlabel('Step')
plt.ylabel('Reward')

plt.subplot(235)
plt.plot(stoch_frequencies)
plt.title('Stochastic Policy Frequencies (Non-Zero)')
plt.xlabel('Step')
plt.ylabel('Frequency')

plt.subplot(236)
plt.plot(stoch_impedances)
plt.title('Stochastic Policy Impedance Slopes (Non-Zero)')
plt.xlabel('Step')
plt.ylabel('Impedance Slope')

plt.tight_layout()
plt.savefig('training_and_testing_results.png')
plt.show()

In [ ]:
# Save training data
training_data = pd.DataFrame({
    'Rewards': callback.rewards,
    'Frequencies': callback.frequencies,
    'Impedances': callback.impedances,
    'Actions': callback.actions,
    'States': callback.states
})
training_data.to_csv('Random_generation_V2.csv', index=False)

# Save stochastic testing data
stochastic_data = pd.DataFrame({
    'Rewards': stoch_rewards,
    'Frequencies': stoch_frequencies,
    'Impedances': stoch_impedances,
    'Actions': stoch_test_actions
})
stochastic_data.to_csv('stochastic_Random_generation_V2.csv', index=False)

print("Data has been saved to 'Random_generation_V2.csv'")

# The rest of the code (plotting and showing) remains the same
plt.tight_layout()
plt.savefig('Random_generation_V2.png')
plt.show()

In [ ]:
# Test function
def test_model_non_zero(model, env, n_steps=20, deterministic=True):
    test_rewards = []
    test_frequencies = []
    test_impedances = []
    test_actions = []  # New list to store actions
    obs = env.reset()
    step_count = 0
    while step_count < n_steps:
        action, _states = model.predict(obs, deterministic=deterministic)
        obs, rewards, dones, info = env.step(action)
        
        # Check if the result is non-zero
        if rewards[0] != -10 :  # Assuming obs[0][0] is frequency and obs[0][1] is impedance slope
            test_rewards.append(rewards[0])
            test_frequencies.append(obs[0][0])
            test_impedances.append(obs[0][1])
            test_actions.append(action[0])  # Store the action
            step_count += 1
        
        if dones:
            obs = env.reset()
    
    return test_rewards, test_frequencies, test_impedances, test_actions

# Test with stochastic policy
stoch_rewards, stoch_frequencies, stoch_impedances, stoch_test_actions = test_model_non_zero(model, env, deterministic=False)

print(stoch_test_actions)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Function to safely convert string to numpy array
def str_to_numpy(s):
    try:
        return np.fromstring(s.strip('[]'), sep=' ')
    except:
        return np.array([])

# Read the CSV files
training_data = pd.read_csv('training_data_50MHz_e.csv')
stochastic_data = pd.read_csv('stochastic_testing_data_50MHz_e.csv')

# Convert string representations of arrays to numpy arrays
training_data['Actions'] = training_data['Actions'].apply(str_to_numpy)
training_data['States'] = training_data['States'].apply(str_to_numpy)
stochastic_data['Actions'] = stochastic_data['Actions'].apply(str_to_numpy)

# Create a figure with subplots
fig, axs = plt.subplots(3, 2, figsize=(15, 20))
fig.suptitle('Training and Stochastic Testing Data', fontsize=16)

# Plot training data
axs[0, 0].plot(training_data['Rewards'])
axs[0, 0].set_title('Training Rewards')
axs[0, 0].set_xlabel('Step')
axs[0, 0].set_ylabel('Reward')

axs[1, 0].plot(training_data['Frequencies'])
axs[1, 0].set_title('Training Frequencies')
axs[1, 0].set_xlabel('Step')
axs[1, 0].set_ylabel('Frequency')

axs[2, 0].plot(training_data['Impedances'])
axs[2, 0].set_title('Training Impedances')
axs[2, 0].set_xlabel('Step')
axs[2, 0].set_ylabel('Impedance')

# Plot stochastic testing data
axs[0, 1].plot(stochastic_data['Rewards'])
axs[0, 1].set_title('Stochastic Testing Rewards')
axs[0, 1].set_xlabel('Step')
axs[0, 1].set_ylabel('Reward')

axs[1, 1].plot(stochastic_data['Frequencies'])
axs[1, 1].set_title('Stochastic Testing Frequencies')
axs[1, 1].set_xlabel('Step')
axs[1, 1].set_ylabel('Frequency')

axs[2, 1].plot(stochastic_data['Impedances'])
axs[2, 1].set_title('Stochastic Testing Impedances')
axs[2, 1].set_xlabel('Step')
axs[2, 1].set_ylabel('Impedance')

# Adjust layout and display the plot
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

# Plot actions
fig, axs = plt.subplots(2, 1, figsize=(15, 10))
fig.suptitle('Actions during Training and Testing', fontsize=16)

# Training actions
action_data = pd.DataFrame(training_data['Actions'].tolist())
action_data.plot(ax=axs[0])
axs[0].set_title('Training Actions')
axs[0].set_xlabel('Step')
axs[0].set_ylabel('Action Value')
axs[0].legend(loc='center left', bbox_to_anchor=(1, 0.5))

# Testing actions
action_data = pd.DataFrame(stochastic_data['Actions'].tolist())
action_data.plot(ax=axs[1])
axs[1].set_title('Stochastic Testing Actions')
axs[1].set_xlabel('Step')
axs[1].set_ylabel('Action Value')
axs[1].legend(loc='center left', bbox_to_anchor=(1, 0.5))

plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

# Plot states (only for training data)
fig, ax = plt.subplots(figsize=(10, 5))
state_data = pd.DataFrame(training_data['States'].tolist())
state_data.plot(ax=ax)
ax.set_title('States during Training')
ax.set_xlabel('Step')
ax.set_ylabel('State Value')
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

plt.tight_layout()
plt.show()

print("Plots have been generated and displayed.")